In [ ]:
#@markdown **Check GPU**
!nvidia-smi

In [ ]:
%pip install -q faster-whisper transformers

In [ ]:
import faster_whisper, math

In [ ]:
#@markdown **SRT extract tools**
def convert_to_hms(seconds: float) -> str:
	hours, remainder = divmod(seconds, 3600)
	minutes, seconds = divmod(remainder, 60)
	milliseconds = math.floor((seconds % 1) * 1000)
	output = f"{int(hours):02}:{int(minutes):02}:{int(seconds):02},{milliseconds:03}"
	return output

def save_srt(segs, filename: str) -> None:
	count = 0
	full_txt = []
	for segment in segs:
		count +=1
		duration = f"{convert_to_hms(segment.start)} --> {convert_to_hms(segment.end)}\n"
		text = f"{segment.text.lstrip()}\n\n"
		full_txt.append(f"{count}\n{duration}{text}")
	with open(filename, mode="w", encoding="UTF-8") as f:
		f.writelines(full_txt)

In [ ]:
#@markdown **Fine-tuned model from huggingface**
!ct2-transformers-converter --model vumichien/whisper-large-v2-mix-jp --output_dir test-ct2 --quantization float16

In [ ]:
model = faster_whisper.WhisperModel("large-v2", device="cuda", compute_type="float16")

In [ ]:
segments, info = model.transcribe("HNDS-048.mp4", language="ja", task="translate", vad_filter=True) # built-in Silero VAD

save_srt(segments, "hnds048.srt")